<a href="https://colab.research.google.com/github/StevenLomon/face-recognition-project/blob/Bashir/face_recognition_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import imghdr
from imutils import paths

In [15]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [24]:
image_paths = list(paths.list_images('/Users/yari/2023/Applicerad_AI/fairfacedata/fairface_images'))

In [8]:
df = pd.read_csv('fairface.csv')

In [4]:
df.head(5)

,file,age,gender,race,phototype
0,1.jpg,50-59,Male,East Asian,III
1,10.jpg,30-39,Male,Middle Eastern,V
2,100.jpg,20-29,Female,East Asian,III
3,1000.jpg,30-39,Male,White,I & II
4,10004.jpg,40-49,Male,Indian,V


In [5]:
df.head(100)

,file,age,gender,race,phototype
0,1.jpg,50-59,Male,East Asian,III
1,10.jpg,30-39,Male,Middle Eastern,V
2,100.jpg,20-29,Female,East Asian,III
3,1000.jpg,30-39,Male,White,I & II
4,10004.jpg,40-49,Male,Indian,V
...,...,...,...,...,...
95,10127.jpg,20-29,Male,White,I & II
96,10128.jpg,30-39,Female,Black,VI
97,10129.jpg,30-39,Male,Black,VI
98,1013.jpg,20-29,Male,East Asian,III
